<a href="https://colab.research.google.com/github/cheolhakja/fine-dust-prediction/blob/main/graduation_preprocess_%EB%AF%B8%EC%84%B8%EB%A8%BC%EC%A7%80%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%95%A9%EC%B9%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_files = [
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_070305.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_072258.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_073204.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_073915.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_095309.csv',
    '/content/drive/MyDrive/graduation_project/air_quality_data_20250702_100842.csv'
]


In [ ]:
# 모든 파일 불러와서 하나의 DataFrame으로 병합
df_all = pd.concat([pd.read_csv(f) for f in data_files], ignore_index=True)

In [ ]:
# 혹시라도 (stationName, dataTime)이 중복되어 있다면 제거
df_all = df_all.drop_duplicates(subset=['stationName', 'dataTime'])


In [ ]:
# 원하는 시간 범위로 필터링

# 1. 문자열 → datetime 변환
df_all['dataTime'] = pd.to_datetime(df_all['dataTime'], errors='coerce')

# 혹시 NaT로 바뀐 row가 있나 확인
print("변환 실패한 row 수:", df_all['dataTime'].isna().sum())

# 2. NaT 제거
df_all = df_all.dropna(subset=['dataTime'])

# 3. 시간 범위 설정
start_time = pd.to_datetime('2025-04-02 21:00')
end_time   = pd.to_datetime('2025-07-02 15:00')

# 4. 시간 필터링
df_filtered = df_all[
    (df_all['dataTime'] >= start_time) &
    (df_all['dataTime'] <= end_time)
]


변환 실패한 row 수: 15137


In [ ]:
# 각 시간대에 측정소 167개씩 있는지 확인
count_by_time = df_filtered.groupby('dataTime')['stationName'].nunique()
print(count_by_time.value_counts())  # 모두 167개여야 함


stationName
167    1216
166     618
164     127
165     105
163      22
Name: count, dtype: int64


In [ ]:
# 구글 드라이브에 저장
df_filtered.to_csv('/content/drive/MyDrive/graduation_project/filtered_air_quality_data.csv', index=False)
